Task3 - Stratified Sampling

Task3 Steps
- (1)Shortest Route Recommendation using Air quality and Taxi mobility Data (All Data).
- Perform Stratified Sampling.
- (2)Shortest Route Recommendation using Air quality and Taxi mobility Data(SampledData).
- Measure time and Accuracy for both routes and compare.

(1)Shortest Route Recommendation using Air quality and Taxi mobility Data (All Data).

Read Air Quality, Taxi Mobility Data and Street Network Data

In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point

# calculating time for Shortest Route Recommendation 
# using Air quality and Taxi mobility Data (All Data).
import time as time
start_cell1 = time.time()

# Read air quality data
pm = "https://raw.githubusercontent.com/IsamAljawarneh/datasets/master/data/NYC_PM.csv"
pmdata = pd.read_csv(pm, engine='python') #contents of data
pmdata.shape[0]
print(pmdata.columns)

Index(['SensorID', 'time', 'latitude', 'longitude', 'bin0', 'bin1', 'bin2',
       'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10',
       'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'bin16', 'bin17', 'bin18',
       'bin19', 'bin20', 'bin21', 'bin22', 'bin23', 'temperature', 'humidity',
       'pm1', 'pm25', 'pm10'],
      dtype='object')


In [2]:
# Read NYC taxi mobility data
NYC_taxi = "https://github.com/IsamAljawarneh/datasets/raw/master/data/nyc1.zip"
trips = pd.read_csv(NYC_taxi)
trips.shape[0]

1445285

In [3]:
#Removing erroneous coordinates (0,0) from the dataset
trips.rename(columns={'Pickup_longitude':'longitude','Pickup_latitude':'latitude'}, inplace=True)
trips = \
trips[(trips['latitude'] != 0 ) & \
(trips['longitude']!=0 )]
trips.shape[0]

1442776

In [4]:
# Read NYC street network data
nyc_polygon = gpd.read_file('https://raw.githubusercontent.com/IsamAljawarneh/datasets/master/data/nyc_polygon.geojson')
nyc_polygon = nyc_polygon.to_crs('EPSG:4326')
G = ox.graph_from_polygon(nyc_polygon.geometry.unary_union, network_type='walk')

- Assigning weights to Graph edges using air quality and Mobility.
- Air quality values at a distance of 40 meters from th edge, are combined for that edge/street only.
- Number of cars values, for each edge/street is summed up, at a distance of 40 meters from the edge/street.
- Greater pm25 value -> More pollution.
- Greater Number of cars -> More traffic, So, more pollution and traffic gives larger edge/street weight in graph, hence not the shortest path.
- So the less weight means less polluton and traffic, hence it will be the shortest path based on Air quality and mobility factors.

In [ ]:

avg_pm25 = pmdata.groupby(['latitude', 'longitude']).agg({'pm25': 'mean'}).reset_index()
# Iterate through nodes and calculate combined weight for edges with geometry information
for u, v, data in G.edges(data=True):
    # Check if the 'geometry' key is present in the data dictionary
    
    if 'geometry' not in data:
        # Skiping edges without geometry information
        continue
    
    pm25 = 0  # Placeholder for air quality parameter
    num_cars = 0  # Placeholder for mobility parameter (number of cars)
    
    # Calculate air quality parameter (pm25) for edge (u, v)
    for index, row in avg_pm25.iterrows():
        point = Point(row['longitude'], row['latitude'])
        if point.distance(data['geometry']) < 0.04:  # 40 meters
            #print(point.distance(data['geometry']))
            pm25 += row['pm25']
    
    # Calculate mobility parameter (number of cars) for edge (u, v)
    for index, row in trips.iterrows():
        pickup_point = Point(row['longitude'], row['latitude'])
        dropoff_point = Point(row['Dropoff_longitude'], row['Dropoff_latitude'])
        if pickup_point.distance(data['geometry']) < 0.04 or dropoff_point.distance(data['geometry']) < 0.04:
            #print (pickup_point.distance(data['geometry']),' + ',dropoff_point.distance(data['geometry']) )
            num_cars += 1
   
    # Calculate combined weight
    data['weight'] = pm25 + num_cars
    print('weight of edges/street ',pm25 + num_cars)
#The code take more than 1 hour to run.

end_cell1 = time.time()
print('Total time using Full population data = ', end_cell1 - start_cell1)

In [ ]:
# Finding nearest nodes to start and destination points
start_point = (40.785091, -73.968285)  # Central Park approximate location
end_point = (40.758896, -73.985130)  # Times Square approximate location
start_node = ox.distance.nearest_nodes(G, X=start_point[1], Y=start_point[0])
end_node = ox.distance.nearest_nodes(G, X=end_point[1], Y=end_point[0])

# Calculate shortest path considering combined weights
shortest_path = nx.shortest_path(G, start_node, end_node, weight='weight')
fig, ax = ox.plot_graph_route(G, shortest_path, route_color='green', route_linewidth=6, node_size=0) 

(2)Shortest Route Recommendation using Air quality and Taxi mobility Data(SampledData)

Adding Geohash attribute to Graph using Graph geometry

In [7]:
start_cell1 = time.time()
# Read NYC street network data
nyc_polygon = gpd.read_file('https://raw.githubusercontent.com/IsamAljawarneh/datasets/master/data/nyc_polygon.geojson')
nyc_polygon = nyc_polygon.to_crs('EPSG:4326')
Gs = ox.graph_from_polygon(nyc_polygon.geometry.unary_union, network_type='walk')

In [15]:
for u, v, data in Gs.edges(data=True):
    if 'geometry' not in data:
        # Skiping edges without geometry information
        continue
    # Assuming the geometry is a LINESTRING
    geometry = data['geometry']
    # Calculate the center of the edge (for better accuracy)
    center = geometry.interpolate(0.5, normalized=True)
    # Calculate the geohash
    geohash_value = gh.encode(center.y, center.x, precision=6)  
    # Add the geohash as an attribute
    data['geohash'] = geohash_value

Adding Geohash in Air quality and Mobility data and Sampling it

In [26]:
import pygeohash as gh

# Configurable parameters
sampling_fraction = 0.1 
geohash_precision = 6

# Convert pmdata to a GeoDataFrame
avg_pm25 = pmdata.groupby(['latitude', 'longitude']).agg({'pm25': 'mean'}).reset_index()
pmdata_geo = gpd.GeoDataFrame(avg_pm25, geometry=gpd.points_from_xy(avg_pm25.longitude, avg_pm25.latitude))
# Perform spatial join between air quality data and street network
pmdata_sjoin = gpd.sjoin(pmdata_geo, nyc_polygon, op='within')
# Add geohash column to the resulting datasets
pmdata_sjoin['geohash'] = pmdata_sjoin.geometry.apply(lambda x: gh.encode(x.y, x.x, precision=geohash_precision))
# Sample air quality data within each geohash
sampled_pmdata = pmdata_sjoin.groupby('geohash').apply(lambda x: x.sample(frac=sampling_fraction))
sampled_pmdata.reset_index(drop=True, inplace=True)  # Reset index after groupby
print('No of Rows left in PMDATA after Stratified sampling = ',sampled_pmdata.shape[0])

# Convert trips data to a GeoDataFrame

trips_geo = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.longitude, trips.latitude))
# Perform spatial join between trip data and street network
trips_sjoin = gpd.sjoin(trips_geo, nyc_polygon, op='within')
# Add geohash column to the resulting trip datasets
trips_sjoin['geohash'] = trips_sjoin.geometry.apply(lambda x: gh.encode(x.y, x.x, precision=geohash_precision))
# Sample trip data within each geohash
sampled_trips = trips_sjoin.groupby('geohash').apply(lambda x: x.sample(frac=sampling_fraction))
sampled_trips.reset_index(drop=True, inplace=True)  # Reset index after groupby
print('No of Rows left in mobility DATA after Stratified sampling = ',sampled_trips.shape[0])

# # Step 5: Integration of Sampled Data

# # Merge sampled air quality data with sampled trip data
joined_data = sampled_pmdata.merge(sampled_trips, on='geohash', how='inner')
print("Joined SAMPLED data",joined_data.shape[0])



c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\Dell\AppData\Local\Temp\ipykernel_10256\1110735428.py:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  pmdata_sjoin = gpd.sjoin(pmdata_geo, nyc_polygon, op='within')


No of Rows left in PMDATA after Stratified sampling =  4728


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\Dell\AppData\Local\Temp\ipykernel_10256\1110735428.py:23: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  trips_sjoin = gpd.sjoin(trips_geo, nyc_polygon, op='within')


No of Rows left in mobility DATA after Stratified sampling =  144109
hi 822389


Calculating Edge/street weight and adding in graph

In [32]:
import geopy.distance
import numpy as np

#removing unnecessary column
sampled_pmdata = sampled_pmdata[['geohash', 'pm25', 'longitude', 'latitude']]
sampled_trips = sampled_trips[['geohash', 'longitude','latitude','Dropoff_longitude','Dropoff_latitude','Trip_distance']]

for u, v, data in Gs.edges(data=True):
    if 'geohash' not in data:
        # Skiping edges without geometry information
        continue
    # Retrieve geohash of the edge
    edge_geohash = data['geohash']
    print(edge_geohash)
    filtered_pmdata = sampled_pmdata[sampled_pmdata['geohash'] == edge_geohash]
    sampled_pm25 = filtered_pmdata['pm25'].mean()

    filtered_trips = sampled_trips[sampled_trips['geohash'] == edge_geohash]
    sampled_no_of_cars = len(filtered_trips)
    # # Look up air quality data and mobility data for the geohash
    # sampled_pm25 = sampled_pmdata.loc[sampled_pmdata['geohash'] == edge_geohash, 'pm25'].mean()
    # sampled_no_of_cars = sampled_trips.loc[sampled_trips['geohash'] == edge_geohash].shape[0]
    
    edge_weight = sampled_pm25 + sampled_no_of_cars
    print(sampled_pm25)
    print(sampled_no_of_cars)
    # Assign the calculated weight to the edge
    # data['weight'] = edge_weight

dr72hc
nan
0
dr72hc
nan
0
dr72hc
nan
0
dr72hc
nan
0
dr5ruw
nan
0
dr5ruw
nan
0
dr72hg
nan
336
dr72h8
nan
0
dr72h8
nan
0
dr72mw
nan
111
dr72mw
nan
111
dr72mw
nan
111
dr72mw
nan
111
dr72mw
nan
111
dr72my
nan
125
dr72h9
nan
0
dr5rst
0.47
0
dr72jj
nan
2682
dr72jj
nan
2682
dr72jj
nan
2682
dr5rsp
nan
0
dr5rsp
nan
0
dr5rsj
nan
0
dr5rsn
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr5rec
nan
0
dr72h9
nan
0
dr5ree
nan
0
dr5ree
nan
0
dr5reg
2.797516129032259
0
dr5ref
0.87
0
dr5ref
0.87
0
dr5ref
0.87
0
dr5ref
0.87
0
dr5rgf
nan
0
dr5rs4
0.8187499999999999
0
dr5rs4
0.8187499999999999
0
dr5rv4
nan
72
dr5rv4
nan
72
dr5rv4
nan
72
dr5rv4
nan
72
dr5rv4
nan
72
dr5rv4
nan
72
dr5rv4
nan
72
dr5ru8
0.15666666666666665
0
dr5ru8
0.15666666666666665
0
dr5ru8
0.15666666666666665
0
dr5rse
0.23500000000000001
0
dr5rse
0.23500000000000001
0
dr5rev
nan
0
dr5rev
nan
0
dr5rev
nan
0
dr5rut
nan
0
dr5rut
nan
0
dr5rut
nan
0
dr5ruh
nan
0
dr5ruh
nan
0
dr5ru5
nan
0
dr72j2
0.

In [ ]:
# Example origin and destination points (latitude, longitude)
start_point = (40.785091, -73.968285)  # Central Park approximate location
end_point = (40.758896, -73.985130)  # Times Square approximate location

# Find nearest nodes to origin and destination points
start_node = ox.distance.nearest_nodes(Gs, X=start_point[1], Y=start_point[0])
end_node = ox.distance.nearest_nodes(Gs, X=end_point[1], Y=end_point[0])

# Calculate shortest path considering air quality and mobility parameters
shortest_path_sample_10 = nx.shortest_path(Gs, start_node, end_node, weight='weight')
end_cell1 = time.time()
fig, ax = ox.plot_graph_route(Gs, shortest_path_sample_10, route_color='green', route_linewidth=6, node_size=0) 

Calculating trajctory Similarity Measure (Euclidean Distance) for different sampling fractions.

In [ ]:
from scipy.spatial.distance import euclidean
# Calculate Euclidean Distance between the Path (FUll population) & Path (Sampling fracton = 10%)
euclidean_dist_10 = euclidean(shortest_path, shortest_path_sample_10)
euclidean_dist_20 = euclidean(shortest_path, shortest_path_sample_20)
euclidean_dist_40 = euclidean(shortest_path, shortest_path_sample_40)
# Print the results
print("Euclidean Distance between paths:", euclidean_dist_10)
print("Euclidean Distance between paths:", euclidean_dist_20)
print("Euclidean Distance between paths:", euclidean_dist_40)

Plotting Similarity Measure with different Sampling fraction

In [ ]:
import matplotlib.pyplot as plt

# Euclidean distances for different sampling fractions
euclidean_distances = [euclidean_dist_10, euclidean_dist_20, euclidean_dist_40]

# Sampling fractions
sampling_fractions = [0.1, 0.2, 0.4]

# Plot the results
plt.plot(sampling_fractions, euclidean_distances, marker='o', linestyle='-')
plt.xlabel('Sampling Fraction')
plt.ylabel('Euclidean Distance')
plt.title('Euclidean Distance between Full Population Path and Sampled Paths')
plt.grid(True)
plt.show()